In [3]:
import snsql
from snsql import Privacy

def privacify(df):
    metadata = {
        '':{
            '': {
                'MyTable': {
                    'max_ids': 1,
                    'row_privacy': True,
                    'patient': {
                        'name': 'patient',
                        'type': 'string'
                    },
                    'value': {
                        'name': 'value',
                        'type': 'int'
                    }
                }
            }
        }
    }

    privacy = Privacy(epsilon=0.5, delta=0.01)
    reader = snsql.from_connection(df, privacy=privacy, metadata=metadata)
    result = reader.execute('SELECT value, COUNT(patient) as amount FROM MyTable GROUP BY value')

    return pd.DataFrame(result[1:], columns = result[0])


In [5]:
import sqlite3

conn = sqlite3.connect("medical.db")
menQuery = """
    SELECT ROUND(AVG(CAST(o.value AS DECIMAL))) as value, p.patient
    FROM observations o
    JOIN patients p ON p.patient = o.patient
    WHERE o.description = 'Body Height' AND p.gender = 'M' AND p.birthdate < '2000'
    GROUP BY o.patient
"""

import pandas as pd
menDf = pd.read_sql_query(menQuery, conn)
print(menDf)
privacify(menDf)

     value                               patient
0    181.0  01290832-65db-488f-9d67-50e3a6d468ea
1    182.0  0310e597-25eb-4412-af50-0cf0ff32b9ba
2    179.0  03412d16-9ffe-4d5f-9014-841055a00bba
3    178.0  04090939-7263-4562-90c7-547d117113f2
4    173.0  0419b689-ddaa-45aa-abdc-64e321cb5c03
..     ...                                   ...
429  179.0  fc5f5d8a-ae4c-4657-a1de-824519739a0d
430  174.0  fc69629e-7bb6-43a5-814c-f8381e348511
431  163.0  fc94c4b3-a114-4c28-a8e8-0f890b674b22
432  190.0  fec1e10f-4478-49d7-bc43-a4bae7b70629
433  177.0  fff429dc-1604-461c-9af0-25c2c9350759

[434 rows x 2 columns]


/Users/lukas/opt/anaconda3/lib/python3.9/site-packages/snsql/sql/private_reader.py:121: UserWarning: Dimension censoring is enabled, with Mechanism.discrete_laplace as the thresholding mechanism. 
This is an unsafe floating point mechanism.  Counts used for censoring will be revealed in any queries that request COUNT DISTINCT(person), 
leading to potential privacy leaks. If your query workload needs to reveal distinct counts of individuals, consider doing the dimension
censoring as a preprocessing step.  See the documentation for more information.
  warnings.warn(


,value,amount
0,151,11
1,163,35
2,165,12
3,168,9
4,170,13
5,171,13
6,172,19
7,173,22
8,174,18
9,175,40
